In [1]:
import json
import boto3
from statistics import mode, StatisticsError

# Função para ler o arquivo JSON do S3
def read_json_from_s3(bucket_name, file_path):
    s3 = boto3.client('s3')
    response = s3.get_object(Bucket=bucket_name, Key=file_path)
    content = response['Body'].read().decode('utf-8')
    return json.loads(content)

# Função para calcular KDA, ouro por minuto e score de visão
def extract_player_stats(match_history):
    kdas = []
    gold_per_minute = []
    vision_scores = []
    
    for match in match_history:
        # print(match['challenges']['kda'])
        kda = match['challenges']['kda']
        kdas.append(kda)

        # Extrair ouro por minuto
        gpm = match['challenges']['goldPerMinute']
        gold_per_minute.append(gpm)

        # Extrair score de visão
        vision_score = match['visionScore']
        vision_scores.append(vision_score)

    return kdas, gold_per_minute, vision_scores

# Função para calcular a moda de uma lista (com tratamento de exceção)
def calculate_mode(stat_list):
    try:
        return mode(stat_list)
    except StatisticsError:
        # Se não houver moda, retorna a média como fallback
        return sum(stat_list) / len(stat_list) if len(stat_list) > 0 else None

# Função para processar o arquivo JSON e retornar uma linha de saída
def process_json(bucket_name, file_path):
    # Ler o arquivo JSON do S3
    match_data = read_json_from_s3(bucket_name, file_path)

    # Para cada jogador, calcular as estatísticas
    output_rows = []

    for player_data in match_data:
        match_history = player_data['matchHistory']

        # Extrair KDA, ouro por minuto e score de visão
        kdas, gpm, vision_scores = extract_player_stats(match_history)

        # Calcular a moda para cada estatística
        kda_mode = calculate_mode(kdas)
        gpm_mode = calculate_mode(gpm)
        vision_score_mode = calculate_mode(vision_scores)

        # Criar a linha de saída
        output_row = [
            kda_mode,
            gpm_mode,
            vision_score_mode
        ]
        
        output_rows.append(output_row)
    
    return output_rows

In [2]:
bucket_name = 'partida-para-prever'
file_path_100 = 'team_100/match_history.json'  # Caminho do arquivo no S3
file_path_200 = 'team_200/match_history.json'
result_100 = process_json(bucket_name, file_path_100)
result_200 = process_json(bucket_name, file_path_200)
    
# Mostrar a saída no formato desejado (como linhas de uma tabela)
# for row in result_azul:
#         print(row)
# for row in result_vermelho:
#         print(row)
# print(result_azul)
# print(result_vermelho)
print(result_100 + result_200)

/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


[[1, 433.30803097675533, 19], [3, 426.58727064897784, 19], [1, 489.9826178896077, 19.25], [2.6666666666666665, 524.2297062100663, 20.0], [3, 326.4970999927794, 39.95], [2.3636363636363638, 471.69953636674893, 4], [2.5, 398.19929456739226, 30.95], [2.75, 476.4634867820679, 27], [3.2046825396825396, 462.6115483421969, 12], [2, 422.936363195574, 21]]
